# Asyncio test

## Inicialization

In [1]:
from comap import api_async
from dotenv import dotenv_values
import asyncio
import aiohttp

In [2]:
secrets = dotenv_values(".env.secret")
shared = dotenv_values(".env.shared")

client_id = secrets['CLIENT_ID']
secret = secrets['SECRET']
comap_key = secrets['COMAP_KEY']
login_id =  secrets['LOGIN_ID']
genset_guid = shared['GENSET_ID']
date_from = '11/28/2023'
date_to = '12/28/2023'
value_guid = 'e6135b58-f4df-4799-a78b-4d27d6fc36b2'

In [3]:
session = aiohttp.ClientSession(raise_for_status=True)

identity = api_async.Identity(session, comap_key)
token = await identity.authenticate(client_id, secret)
token

{'token_type': 'Bearer',
 'expires_in': 3599,
 'ext_expires_in': 3599,
 'access_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6IlQxU3QtZExUdnlXUmd4Ql82NzZ1OGtyWFMtSSJ9.eyJhdWQiOiI3NjJlNWQ5NS0wNWMxLTQ0OWQtOGU5Zi1jNmIzMjk1ZDhkYzQiLCJpc3MiOiJodHRwczovL2xvZ2luLm1pY3Jvc29mdG9ubGluZS5jb20vYjdhNWE1NTItNjQyNy00MzA5LThiZmMtNTE1NGU3ZDczNDYzL3YyLjAiLCJpYXQiOjE3MDE1MjI3NzEsIm5iZiI6MTcwMTUyMjc3MSwiZXhwIjoxNzAxNTI2NjcxLCJhaW8iOiJFMlZnWUpoM05tdVNsZmZ5YlU2dTFqSFZkZDh1L043Q2Qybm1ucWpxZ21ySEVINnhlUW9BIiwiYXpwIjoiNzFmNmRmYmYtNjY2MS00N2JlLWI3M2UtYWJlY2I2ZjEzNWExIiwiYXpwYWNyIjoiMSIsInJoIjoiMC5BU0lBVXFXbHR5ZGtDVU9MX0ZGVTU5YzBZNVZkTG5iQkJaMUVqcF9Hc3lsZGpjUWlBQUEuIiwidGlkIjoiYjdhNWE1NTItNjQyNy00MzA5LThiZmMtNTE1NGU3ZDczNDYzIiwidXRpIjoiTDFuNDlFZzhqRWl1M1ptRl94MERBQSIsInZlciI6IjIuMCJ9.EDqEEnvIZZbZuv5tf6wH78nE4YSYcK7RjmGcqcgpEEe6ZhivZ4t45pZkC5bikQJ90KMzWI9l51kbydhRTUljSMqZB2Kl25bITfSPzaszDERLumUNaXiugsVTBAwOKZX5OMw_2lyOeMCsv1LnnQo6IPSlIi95sTe0xJ5usTPGGyoBK2iOLsBxtaVBoVJpg4xb-hUys2gcCr_0wwDdyGhIIdu1ejFtnXH_o

In [4]:
wsv = api_async.WSV(session, login_id, comap_key, token['access_token'])

wsv_units = await wsv.units()

## Units

In [5]:
for unit in await wsv.units():
    print(unit["name"])

Hybrid Barcelona
IL-NT
ID FLX lite
Cogeneration unit ABB Prague
Westport 
ID-Lite
IA-NT STD test
IC-NT Mint 2
Cogeneration unit Hilton London
ID-EM
Harvester Goa
Habana hospital
AIO GAS
New marine
IL9
Controller Solo
Hybrid
test33
sifrovani IL3
Global
C07 - Hybrid Master
IG200_VAJL
HYBRID ComAp headquarter Prague


## Info

In [6]:
info = await wsv.info(genset_guid)
values = [('Name', info['name']),
          ('Owner', info['ownerLoginId']),
          ('Timezone', info['timezone']),
          ('Position', f'{info["position"]["latitude"]},{info["position"]["longitude"]}')
]
for desc, value in values:
    print(f'{desc:10}{value}')

Name      Westport 
Owner     StaceyH5110
Timezone  America/New_York
Position  40.768301,-73.989698


## Values

In [7]:
from comap.constants import VALUE_GUID
values = await wsv.values(
    genset_guid,
    f'{VALUE_GUID["mode"]}, '
    f'{VALUE_GUID["engine_state"]}'
)
for value in values:
    print(f'{value["name"]:<16} : {value["value"]}')

ControllerMode   : AUT


## History

In [8]:
history = await wsv.history(
    genset_guid,
    date_from,
    date_to,
    VALUE_GUID["engine_state"]
)
for event in history[0]["history"]:
    print(f'{event["value"]:<8} valid to {event["validTo"]}')

Loaded   valid to 2023-12-02 13:00:00+00:00
Loaded   valid to 2023-12-02 14:00:00+00:00


## Files

In [9]:
for file in await wsv.files(genset_guid):
    print(f'{file["generated"]} {file["fileType"]} [{file["fileName"]}]')

2018-01-24 06:00:10+00:00 unitHistory [2018-01-24_01-00_Westport .csv]
2018-01-25 06:00:11+00:00 unitHistory [2018-01-25_01-00_Westport .csv]
2018-01-26 06:00:10+00:00 unitHistory [2018-01-26_01-00_Westport .csv]
2018-01-27 06:00:11+00:00 unitHistory [2018-01-27_01-00_Westport .csv]
2018-01-28 06:00:12+00:00 unitHistory [2018-01-28_01-00_Westport .csv]
2018-01-29 06:00:11+00:00 unitHistory [2018-01-29_01-00_Westport .csv]
2018-01-30 06:00:10+00:00 unitHistory [2018-01-30_01-00_Westport .csv]
2018-01-31 06:00:11+00:00 unitHistory [2018-01-31_01-00_Westport .csv]
2018-02-01 06:00:10+00:00 unitHistory [2018-02-01_01-00_Westport .csv]
2018-02-02 06:00:11+00:00 unitHistory [2018-02-02_01-00_Westport .csv]
2018-02-02 16:05:06+00:00 unitHistory [2018-02-02_11-04_Westport .csv]
2018-02-03 06:00:11+00:00 unitHistory [2018-02-03_01-00_Westport .csv]
2018-02-04 06:00:11+00:00 unitHistory [2018-02-04_01-00_Westport .csv]
2018-02-05 06:00:11+00:00 unitHistory [2018-02-05_01-00_Westport .csv]
2018-0

## Comments

In [10]:
for comment in await wsv.comments(genset_guid):
    print(f'{comment["author"]:<18} {comment["date"]} {comment["text"]}')

Stacey Hoppe       2018-08-17 20:27:43.124000+00:00 8&#x2F;17&#x2F;18- unit down inverter jesse will be at site 8&#x2F;18 to reset &#x28;sh&#x29;
Stacey Hoppe       2018-08-30 18:09:25.925000+00:00 8&#x2F;30&#x2F;2018- Unit down till 8&#x2F;31&#x2F;2018 FLS EngWtr out A, STP EngWtrOUtAhigh
Stacey Hoppe       2019-09-03 19:44:27.009000+00:00 unit down exhaust flex broken
Stacey Hoppe       2019-10-15 19:42:45.736000+00:00 Unit will be down till tomorrow need to investigate Cyl # head  &#x28;SH&#x29;
Stacey Hoppe       2019-12-02 19:01:01.698000+00:00 Unit will be down for 20,000 hour service
Stacey Hoppe       2020-09-08 22:01:36.014000+00:00 Inverter fault shut unit down
Stacey Hoppe       2020-09-23 17:26:23.423000+00:00 BOC Inverter Shutdown. can restart unit
Stacey Hoppe       2020-09-25 09:30:13.967000+00:00 Inverter fault


## Command

In [11]:
if await wsv.command(genset_guid,'start'):
    print('Genset start succesful')
else:
    print('Genset start failed')

Genset start succesful


## Close session

In [12]:
await session.close()